In [20]:
import pandas as pd
import numpy as np
import networkx as nx
import csv
from multiprocessing import Pool
import time

In [2]:
train = pd.read_csv('./train_data.csv',
                header=0, sep=',',
                infer_datetime_format=True)
test = pd.read_csv('./test_data.csv',
                  header=0, sep=',',
                  infer_datetime_format=True)
train.head(20)

,video_id,uploader,age,category,length,views,rate,ratings,comments,1,...,11,12,13,14,15,16,17,18,19,20
0,LKh7zAJ4nwo,TheReceptionist,653.0,Entertainment,424.0,13021.0,4.34,1305.0,744.0,NaN,...,DjdA-5oKYFQ,vmdPOOd6cxI,uX81lMev6_o,NaN,W1Uo5DQTtzc,heqocRij5P0,2ZgXx72XmoE,1tUDzOp10pk,DH56yrIO5nI,E-3zXq_r4w0
1,7D0Mf4Kn4Xk,periurban,583.0,Music,201.0,6508.0,4.19,687.0,312.0,AvSK0qPw7EU,...,e2k0h6tPvGc,NaN,VKFqqoeMdjw,NSzrwv5MCwc,y3IDp2n7B48,N1lkLaLJHlc,NHB0a0xtLgU,Eq4hGkIqBGw,bdDskrr8jRY,JngPWhfCb2M
2,n1cEq1C8oqQ,Pipistrello,525.0,Comedy,125.0,1687.0,4.01,363.0,141.0,YfShxdbAJS8,...,eprHhmurMHg,yLup8wjbSIo,z5wDjq8o60c,bWlPSLUT-6U,bSPVayE0NhE,F2_5KOnSsfI,dsBTo5LExr0,gnpvEvuiFoQ,3nH5Tccz8EQ,sEqCkwPmQ_w
3,OHkEzL4Unck,ichannel,638.0,Comedy,299.0,8043.0,4.40,518.0,371.0,NaN,...,eyUSTmEUQRg,LQUV_XGzHmA,nfBfC8bif1Y,aiYwo5K0VWg,N4EYgXReBzM,FMuWYExDEJk,Ml2NaXU6gms,DVNwUKAuB3I,mDjwzhc8dQ0,NyC_0Z6zoUk
4,-boOvAGNKUc,mrpitifulband,639.0,Music,287.0,7548.0,4.48,606.0,386.0,JsdCu9T47iY,...,fmUwUURgsX0,eV2SdBITv8k,sf-Ym_pFP6U,0vhVZQEzgcU,lhWk9SXUjWI,geiABCqmQ84,dDhCZVQf9po,RZo5MisSTWo,saBmFpuwmKA,aVhSaa6aAOg
5,hFFH8DaOHQg,istothehalfabee,592.0,Music,286.0,1759.0,4.45,539.0,244.0,GUgJKzEmsYI,...,hFFH8DaOHQg,4qf7RSNCg40,oDIIOV4VKlA,vEtM1q6gm9Q,NaN,NaN,r89-fFx_tHU,D6ABDEdhQLA,7TyH0ipgdtY,jqNs_S0n7P8
6,LzHjIj3fpR8,Xelanderthomas,686.0,Comedy,168.0,4545.0,4.58,273.0,167.0,PlPynB10vP0,...,NaN,Y_59kWK5W3s,vta4RfQ2Z-I,vWGA5iYgAOU,4WwVOWIqE80,kdZk1Wk7kSw,NaN,4ALe2z---e0,NaN,Qeeq5OoLGJ0
7,SDNkMu8ZT68,w00dy911,630.0,People & Blogs,186.0,10181.0,3.49,494.0,257.0,NaN,...,rjnbgpPJUks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PkGUU_ggO3k,theresident,704.0,Entertainment,262.0,11235.0,3.85,247.0,280.0,K7unV366Qr4,...,PkGUU_ggO3k,fbq2-jd5Dto,p0lq5-8IDqY,AfBxANiGnnU,gVSzbvFnVRY,NaN,RyWz8hwGbY4,mh_MGyx9tgc,JO1LTIFOkTw,l9NJ04JiZj4
9,RX24KLBhwMI,lemonette,697.0,People & Blogs,512.0,24149.0,4.22,315.0,474.0,NaN,...,t60tW0WevkE,dGM3k_4cNhE,2cKd9ERh5-8,EGWutOjVx4M,sfG2rtAkAcg,9BBu5N0iFBg,KVkseZR5coU,3uKOSjE79YA,MwynZ8qTwXA,j72VLPwzd_c


In [3]:
t_wh = list(set(list(train['%d'%i] for i in range(1,21))[0]))
t_wh

[nan,
 'JyUUifBZImQ',
 'apeYJJ5HHmY',
 'kIaBThV1ekc',
 '9Zem0Q5w4sE',
 'm1mu-zdA1Ew',
 '_vRWqmyP2gM',
 'LFGtDeH6lRI',
 'ewsgJXtvCy4',
 'IzWbn88LYBM',
 'g9iOugSinvk',
 'NDxRDck2Ii4',
 'CS7n9INBZzs',
 'layeGD0ziIk',
 'mljdpRp41I4',
 'KE6lzcCnN5s',
 '3LPseigyocg',
 'BePTWzKY0KI',
 'k4hxYFDxCPQ',
 'IHqDvCPuuSI',
 'MDiE7Qfy_bY',
 'S61O4vcOg2k',
 'Hy4_7jSh_2U',
 'kGA7MEPfO-4',
 'a4mBc9eBNX4',
 '1oOPfPK-PPs',
 'GzhKjuU7WcU',
 'TIkolBqK2qA',
 'ik4HBZS2-L8',
 'zqZa1_dER-c',
 'jpAWQW6-pgo',
 'hPDS43jxtik',
 'Pg0_R4_xHbM',
 '9grJcL4hFXw',
 'gCRMzm6t-Ck',
 'PbOLVTAW-mw',
 'b-2pjff7zT8',
 'E1NDJ3faZiE',
 'vDTV1ZeRBJw',
 '19YO6fQUVEk',
 '4dQ53UfXtpM',
 'BNeLNWEeXQ8',
 '3I-hpDJFSGo',
 'UhZYcnjVr3Y',
 'HhFMD6ybP2E',
 'VnYwbcxUOcM',
 '2_a1yo4jyCM',
 'Px3v6xvE6Ek',
 '1NTB3-2pDxw',
 '3H6T70wzWq8',
 'VAaM3tqgDFk',
 'XhjKzk43spo',
 'GtZZsiSRRGY',
 'w9fuY7-JJ8w',
 'mREshcQRsvQ',
 'AarwKuHVPI8',
 '02VsyjNHoCQ',
 '8GGUxVhI43o',
 'nG-lmhl7ojg',
 'Qz3AxRyiHwg',
 'X-z3ZifzaC4',
 '64dUqMxxyjQ',
 'HUP8ng0wNrY',
 '

In [4]:
l = 0
for tmp in list(test['target']):
    if tmp not in list(train['video_id']):
        l = l + 1
l

72506

In [4]:
edge = [0] * 100000
#part 1: if the edge exists in train_data, label that edge 1
for idx_test, video in enumerate(list(test['source'])):
    idx_train = list(train['video_id']).index(video)
    lst_tmp = list(train['%d'%i][idx_train] for i in range(1,21))
    if list(test['target'])[idx_test] in lst_tmp:
        edge[idx_test] = 1

In [96]:
#part 2: if the target video exist in train[video_id] and shares the same uploader with source video, label that edge 1
for idx_test, video in enumerate(list(test['source'])):
    if edge_use[idx_test] == 0:
        if list(test['target'])[idx_test] in list(train['video_id']):
            idx_train = list(train['video_id']).index(video)
            uper = list(train['uploader'])[idx_train]
            idx_test_t = list(train['video_id']).index(list(test['target'])[idx_test])
            if list(train['uploader'])[idx_test_t] == uper:
                edge_use[idx_test] = 1

In [23]:
#part 3: if the target video exist in train[video_id] and shares the same category with source video 
#and the target video's rate is higher than source video, label that edge 1
for idx_test, video in enumerate(list(test['source'])):
    if edge_use[idx_test] == 0:
        if list(test['target'])[idx_test] in list(train['video_id']):
            idx_train = list(train['video_id']).index(video)
            cate = list(train['category'])[idx_train]
            idx_test_t = list(train['video_id']).index(list(test['target'])[idx_test])
            if list(train['category'])[idx_test_t] == cate and list(train['rate'])[idx_test_t] >= list(train['rate'])[idx_train]:
                edge_use[idx_test] = 1

In [26]:
#part 4: if the target video exist in train, use shorttest parth to label edge:
#form graph
graph = {}
for idx, tmp in enumerate(list(train['video_id'])):
    graph[tmp] = list(train['%d'%i][idx] for i in range(1,21) if not pd.isnull(train['%d'%i][idx]))

In [32]:
graph2 = nx.DiGraph(graph)

In [33]:
for node in graph2:
    lin = graph2.neighbors(node)
    for tmp in lin:
        if not graph2.has_edge(tmp, node):
            graph2.add_edge(tmp, node, weight=1.2)

In [ ]:
#shortest path
s = time.time()
shtst = [0] * 100000
edge_use = [0] * 100000
for idx_test, video in enumerate(test['source']):
    if edge_use[idx_test] == 0:
        if test['target'][idx_test] in graph2:
            try:
                shtst[idx_test] = nx.shortest_path_length(graph2, source=video, target=test['target'][idx_test],weight='weight')
            except:
                shtst[idx_test] = 1000
        if not test['target'][idx_test] in graph2:
            shtst[idx_test] = 0
for i,value in enumerate(shtst):
    if edge_use[i] == 0:
        if value < 4:
            edge_use[i] = 1
print(time.time()-s)

In [25]:
edge_id = [i for i in range(100000)]
with open('result.csv', 'w', newline='') as f:
    out = csv.writer(f, delimiter=',')
    out.writerow(['edge_id','edge_present'])
    for i in range(100000):
        out.writerow(['%d'%i,'%d'%edge_use[i]])


In [29]:
sum(edge_use)

46837